# Train U-Net for cervix/os detection

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [35]:
import matplotlib.pylab as plt
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Run on local machine

In [2]:
import os
import datetime

import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 

from keras.callbacks import ModelCheckpoint

# Project
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'common')))
from data_utils import type_1_ids, type_2_ids, type_3_ids, test_ids
from data_utils import RESOURCES_PATH, GENERATED_DATA, get_annotations
from training_utils import get_trainval_id_type_lists2, get_test_id_type_list2
from image_utils import get_image_data
from metrics import logloss_mc
from unet_keras_v1 import get_unet

# Local keras-contrib:
from preprocessing.image.generators import ImageMaskGenerator
from preprocessing.image.iterators import XYIterator

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


In [3]:
np.random.seed(2017)

def xy_provider(image_id_type_list, 
                image_size=(224, 224), 
                test_mode=False,
                verbose=0):        
    while True:
        for i, (image_id, image_type) in enumerate(image_id_type_list):
            if verbose > 0:
                print("Image id/type:", image_id, image_type, "| counter=", counter)

            img = get_image_data(image_id, image_type)
            if img.dtype.kind is not 'u':
                if verbose > 0:
                    print("Image is corrupted. Id/Type:", image_id, image_type)
                continue
            img = cv2.resize(img, dsize=image_size[::-1])
            img = img.transpose([2, 0, 1])
            img = img.astype(np.float32) / 255.0

            label = get_image_data(image_id + "_" + image_type, "label")
            label = cv2.resize(label, dsize=image_size[::-1])
            label = label.transpose([2, 0, 1])

            if test_mode:
                yield img, label, (image_id, image_type)
            else:
                yield img, label
                
        if test_mode:
            return


In [39]:
def train(model, train_id_type_list, val_id_type_list, batch_size=16, nb_epochs=10, image_size=(224, 224)):
    
    samples_per_epoch = (1024 // batch_size) * batch_size
    nb_val_samples = (256 // batch_size) * batch_size
    #samples_per_epoch = (2048 // batch_size) * batch_size
    #nb_val_samples = (1024 // batch_size) * batch_size

    if not os.path.exists('weights'):
        os.mkdir('weights')

    weights_filename = os.path.join("weights", "unet_os_cervix_detector_{epoch:02d}-{val_loss:.4f}.h5")
    model_checkpoint = ModelCheckpoint(weights_filename, monitor='loss', save_best_only=True)

    print("Training parameters: ", batch_size, nb_epochs, samples_per_epoch, nb_val_samples)
    
    train_gen = ImageMaskGenerator(featurewise_center=True,
                                   featurewise_std_normalization=True,
                                   rotation_range=90., 
                                   width_shift_range=0.15, height_shift_range=0.15,
                                   shear_range=3.14/6.0,
                                   zoom_range=0.25,
                                   channel_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True)
    val_gen = ImageMaskGenerator(rotation_range=90., 
                                 horizontal_flip=True,
                                 vertical_flip=True)
    
    train_gen.fit(xy_provider(train_id_type_list, test_mode=True),
                  len(train_id_type_list), 
                  augment=True, 
                  save_to_dir=GENERATED_DATA,
                  save_prefix='os_cervix',
                  batch_size=4,
                  verbose=1)
   
    history = model.fit_generator(
        train_gen.flow(xy_provider(train_id_type_list), 
                       len(train_id_type_list),
                       batch_size=batch_size),
        samples_per_epoch=samples_per_epoch,
        nb_epoch=nb_epochs,
        validation_data=val_gen.flow(xy_provider(val_id_type_list), 
                       len(val_id_type_list),
                       batch_size=batch_size),
        nb_val_samples=nb_val_samples,
        callbacks=[model_checkpoint],
        verbose=1,
    )

    return history

In [5]:
def validate(model, val_id_type_list, batch_size=16, image_size=(224, 224)):
      
    val_iter = XYIterator(xy_provider(val_id_type_list, test_mode=True), 
                          len(val_id_type_list), 
                          None, # image generator
                          batch_size=batch_size,
                          data_format='channels_first')
    
    total_loss = 0.0
    total_counter = 0 
    for x, y_true, info in val_iter:           
        s = y_true.shape[0]
        total_counter += s
        y_pred = model.predict(x)
        loss = logloss_mc(y_true, y_pred)
        total_loss += s * loss
        print("--", total_counter, "batch loss : ", loss)

    if total_counter == 0:
        total_counter += 1

    total_loss *= 1.0 / total_counter   
    print("Total loss : ", total_loss)
    

In [6]:
def predict(model, test_id_type_list, batch_size=16, image_size=(224, 224), info=''):

    
    test_iter = XYIterator(xy_provider(val_id_type_list, test_mode=True), 
                          len(val_id_type_list), 
                          None, # image generator
                          batch_size=batch_size,
                          data_format='channels_first')

    
    df = pd.DataFrame(columns=['image_name', 'image_type', 'os', 'cervix'])
    total_counter = 0
    for x, _, info image_ids in test_iter:            
        y_pred = model.predict(y)    
        s = X.shape[0]
        total_counter += s
        print("--", total_counter)
        for i in range(s):
            df.loc[total_counter + i, :] = (image_ids[i] + '.jpg', ) + tuple(Y_pred[i, :])

    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    df.to_csv(sub_file, index=False)

In [37]:
print("\n {} - Get train/val lists ...".format(datetime.datetime.now()))        
sloth_annotations_filename = os.path.join(RESOURCES_PATH, 'cervix_os.json')
annotations = get_annotations(sloth_annotations_filename)

train_id_type_list, val_id_type_list = get_trainval_id_type_lists2(annotations=annotations, val_split=0.25)

print "Total : %s, Train : %s, Val : %s" % (len(annotations), len(train_id_type_list), len(val_id_type_list))

print("\n {} - Get U-Net model ...".format(datetime.datetime.now()))
unet = get_unet(input_shape=(3, 224, 224), n_classes=2)


 2017-03-28 22:39:13.798069 - Get train/val lists ...
Total : 208, Train : 156, Val : 52

 2017-03-28 22:39:13.809415 - Get U-Net model ...


In [8]:
unet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 224, 224)  896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 32, 224, 224)  128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 224, 224)  0           batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [ ]:
nb_epochs = 50
batch_size = 4   

print("\n {} - Start training ...".format(datetime.datetime.now()))
train(unet, train_id_type_list, val_id_type_list, nb_epochs=nb_epochs, batch_size=batch_size)


 2017-03-28 22:48:13.662957 - Start training ...
('Training parameters: ', 4, 50, 1024, 256)
Load existing file: /Users/vfomin/Documents/ML/Kaggle/Intel_MobileODT/input/generated/os_cervix_stats.npz
No need to recompute statistics
Epoch 1/50
 272/1024 [======>.......................] - ETA: 547s - loss: 0.6010 - jaccard_index: 0.3990 - recall: 0.8244 - precision: 0.6302

In [ ]:
#print("\n {} - Start validation ...".format(datetime.datetime.now()))
#validate(unet, val_id_type_list, batch_size=batch_size)

In [ ]:
#test_id_type_list = get_test_id_type_list2(annotations)
#print("\n {} - Start predictions and write detections".format(datetime.datetime.now()))
#predict(unet, info='unet_no_additional', batch_size=batch_size)
#print("\n {} - Scripted finished".format(datetime.datetime.now()))

## Submit job with `qsub`

In [8]:
from qsub_utils import submit_job
from qsub_utils import setup_configuration
from qsub_utils import PBS_CONFIGURATION

In [9]:
setup_configuration(nodes='1:knl7210:ram96gb')

In [10]:
import time 

unet_cervix_os_detection_with_keras_cmd = [
    "python",
    os.path.abspath(os.path.join("..", "scripts", "unet_cervix_os_detection_with_keras.py"))
]

process, job_info = submit_job(unet_cervix_os_detection_with_keras_cmd, 
                               name='unet_cervix_os_detection', 
                               cwd=RESOURCES_PATH)

try:
    while True:
        out = process.stdout.readline()    
        if len(out) > 0:        
            print out

        if process.poll() is not None and len(out) == 0:
            break
except KeyboardInterrupt:
    !qdel {job_info['id']}
    time.sleep(1.0)
    !qstat

OSError: [Errno 2] No such file or directory

In [48]:
!qstat

In [11]:
#!{" ".join(unet_cervix_os_detection_with_keras_cmd)}

Using Theano backend.
Couldn't import dot_parser, loading of dot files will not be possible.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 0 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

Traceback (most recent call last):
  File "/Users/vfomin/Documents/ML/Kaggle/Intel_MobileODT/scripts/unet_cervix_os_detection_with_keras.py", line 8, in <module>
    from keras.callbacks import ModelCheckpoint
  File "/usr/local/lib/python2.7/site-packages/keras/__init__.py", line 2, in <module>
    from . import backend
  File "/usr/local/lib/python2.7/site-packages/keras/backend/__init__.py", line 64, in <module>
    from .theano_back